# ATP Synthase Model Attempt 3
7.16.2020<br>
In this notebook, we will try to ADD GENE EXPRESSION and use Zoila's data as well.

In [2]:
# Import cell (refined)
import numpy as np
import pandas as pd 

import scipy.integrate
from scipy.integrate import odeint

from biocrnpyler import *

import colorcet

import bokeh.io
import bokeh.plotting
from bokeh.layouts import row

bokeh.io.output_notebook()

Loading BokehJS ...

## ONLY Model Membrane Integration
### BioCRNPyler

----
``` 
DNA --> (k4) RNA
RNA --> (k5) ATP Synthase (protein folding)
ATP Synthase_free --> (k6) ATP Synthase_bound

```
------


In [3]:
k4_tx = 1
k5_tl = 1
k6_mem = 1


dna = Species('dna')
rna = Species('rna')
atpase_f = Species('atpase_f')
atpase_b = Species('atpase_b')

species = [dna, rna, atpase_f, atpase_b]

rxn1 = Reaction([dna], [rna], propensity_type = 'massaction', k = k4_tx, k_rev = 1e-5)
rxn2 = Reaction([rna], [atpase_f], propensity_type = 'massaction', k = k5_tl, k_rev = 1e-5)
rxn3 = Reaction([atpase_f], [atpase_b], propensity_type = 'massaction', k = k6_mem, k_rev = 1e-5)

reactions = [rxn1, rxn2, rxn3]

CRN_membrane_integration = ChemicalReactionNetwork(species = species, reactions = reactions)

x0_dict = {'dna':1}

timepoints = np.linspace(0,20,1000)

re = CRN_membrane_integration.simulate_with_bioscrape(timepoints, x0_dict)

In [4]:
palette = colorcet.b_glasbey_category10

p3 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",
    title = 'Membrane Integration'
        )

p3.line(timepoints, re['dna'], line_width = 2, color = palette[0], legend_label = 'dna' )
p3.line(timepoints, re['rna'], line_width = 2,color = palette[1], legend_label = 'rna' )
p3.line(timepoints, re['atpase_f'], line_width = 2,color = palette[2], legend_label = 'atpase_f' )
p3.line(timepoints, re['atpase_b'], line_width = 2,color = palette[3], legend_label = 'atpase_b' )

bokeh.io.show(p3)

## Model Membrane Integration + ATP Production
### scipy.odeint


----
``` 
DNA --> (k4_tx) RNA
RNA --> (k5_tl) ATP Synthase (protein folding)
ATP Synthase_free --> (k6_mem) ATP Synthase_bound

```
------
```
if ph outside <= 8:
    ADP + Pi + 4 H+ --> (k1) ATP
    ATP --> (k2) ADP + Pi
    outside_H --> (k3) H+
    
elif ph outside > 8:
    ATP + 4 H + --> (k1) ADP + Pi
    H+ --> (k3) outside_H
    
```
----


In [5]:
def rhs(var, t, k1, k2, k3, k4_tx, k5_tl, k6_mem):
    
    atp,adp,pi,h,h20,outside_H,dna,rna,atpase_f,atpase_b = var
    
    #print(atpase_b)
    
    ddna_dt = -k4_tx * dna
    
    drna_dt = k4_tx * dna - k5_tl * rna
    
    datpase_f_dt = k5_tl * rna - k6_mem * atpase_f
    
    datpase_b_dt = k6_mem * atpase_f
    
    
    
    # PH CONTROL PORTION WHEN PH IS GREATER THAN 8, ATP SYNTHASE RUNS BACKWARDS (assume has same rxn rate:(
    ph = np.log10(outside_H)
    if ph > 8 and atpase_b > 0.9:
        #print(-np.log(outside_H))
        
        datp_dt = -k1 * atp * h**4
    
        dadp_dt = k1 * atp * h**4
    
        dpi_dt = k1 * atp * h**4
    
        dh_dt = -k3 * h
        
        dh20_dt = 0

        doutsideH_dt = H_control(var,t,k3, negative = False)
        print("HERE")
      
        
    elif ph <= 8 and atpase_b > 0.9:
       # print('yes')
        
        datp_dt = k1 * adp * pi * h**3.3 - k2 * atp

        dadp_dt = -k1 * adp * pi * h**3.3 + k2 * atp

        dpi_dt = dadp_dt

        dh_dt = k3 * outside_H - k1 * adp * pi * h**4

        dh20_dt = k1 * adp * pi * h**4

        doutsideH_dt = H_control(var, t, k3, negative = True)
        
    else:
        datp_dt = 0
        
        dadp_dt = 0
        
        dpi_dt = 0
        
        dh_dt = 0
        
        dh20_dt = 0
        
        doutsideH_dt = 0
    
    return np.array([datp_dt, dadp_dt, dpi_dt, dh_dt, dh20_dt, doutsideH_dt,
                     ddna_dt, drna_dt, datpase_f_dt, datpase_b_dt])
    


In [6]:
def H_control(var, t, k3, negative):
    
    atp,adp,pi,h,h20,outside_H,dna,rna,atpase_f,atpase_b = var

    
    if t < 7:
        outside_H = 0

    elif t > 7 and t <= 7.2:
        outside_H = 30
    
    elif t > 20 and t <= 20.4:
        outside_H = 30
        
    elif negative == True: # synthesis
        outside_H = -k3 * outside_H
        
    elif negative == False: # hydrolysis
        outside_H = k3 * outside_H



    return outside_H


In [7]:
def rhs_noatpsyn(var,t,k2):
    
    atp,adp,pi = var

#     ddna_dt = -k4 * dna
    
#     drna_dt = k4 * dna - k5 * rna
    
#     datpase_f_dt = k5 * rna - k6 * atpase_f
    
#     datpase_b_dt = k6 * atpase_f
    
    
    datp_dt = -k2 * atp
    
    dadp_dt = k2 * atp
    
    dpi_dt = k2 * atp
    
    return np.array([datp_dt, dadp_dt, dpi_dt])

Some rate justifications: 
- k1 = ATP synthesis rate is > 100 ATP/sec (bionum ID 111975) or 6,000 ATP/min or 360,000 ATP/hr <br>
- k2 = ATP Use rate in Tx/Tl? - we normally assume there is none by hour 20<br>
- k3 = Each ATP requires the transport or 3.33 protons, we round to 4. <br>
- what is k3: outside H to H rate: if 3.33 h+ atp/second and 100 atp/s then 400 h+/sec or 24000 h+ / min or 1,440,000 h+ /hr
- k4: transcription rate (dna to rna)
- k5: translation rate (rna to protein with folding)
- k6: free to bound protein

In [12]:
t_vals = np.linspace(0,75,201)
# k1 = 100 * 60 * 60 # per sec to hr
# k2 = 100 * 60 * 60
# k3 = 333 * 60 * 60 # per sec to hr

k1 = 0.1 # per sec to hr
k2 = 0.2
k3 = 0.1 # per sec to hr
k4_tx = 1
k5_tl = 1
k6_mem = 1

x_args = (k1, k2, k3, k4_tx, k5_tl, k6_mem)

atp_0 = 30
adp_0 = 0
pi_0 = 0
h_0 = 0
h20_0 = 0 
outside_H = 1.5
dna = 1
rna = 0
atpase_f = 0
atpase_b = 0
y0 = np.array([atp_0, adp_0, pi_0, h_0, h20_0, outside_H, dna, rna, atpase_f, atpase_b])

x = odeint(rhs, y0, t_vals, args = x_args)


#colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02']
colors = colorcet.b_glasbey_category10


p = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",
    title = 'All Species with ATP Synthase'
        )
p1 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",title = 'Outside H+'
        )

p2 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",title = 'No ATP Synthase Control'
        )


p3 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",title = 'Membrane Integration Components'
        )

#colors = ['#e7298a','#7570b3','#d95f02','#1b9e77','#66a61e','#e6ab02']
spec_list = ['atp', 'adp', 'pi', 'h+', 'h20', 'outside h+']

for i,spec in enumerate(spec_list):
    if spec_list[i] == 'h20':
        i+=1
    else:
        p.line(t_vals, x[:,i], line_width = 2, color = palette[i], legend_label = spec)
 
spec_list2 = ['dna', 'rna','atpase_f', 'atpase_b']
for i,spec in enumerate(spec_list2):
    p3.line(t_vals, x[:, i+6], line_width = 2, color = palette[i+6], legend_label = spec)


p1.line(t_vals, x[:,5], line_width = 2, color= palette[5] )
        
        
p.legend.click_policy = 'hide'


# Make a plot for use without the ATP synthase
noatpsyn_spec_list = ['atp', 'adp', 'pi']
noatpsyn_0 = np.array([atp_0, adp_0, pi_0])
noatpsyn_args = (k2,)

noatp = odeint(rhs_noatpsyn, noatpsyn_0, t_vals, args = noatpsyn_args)

for i,spec in enumerate(noatpsyn_spec_list):
    p2.line(t_vals, noatp[:,i], line_width = 2, color = palette[i], legend_label = spec)

bokeh.io.show(row(p, p1))
bokeh.io.show(row(p2, p3))



Ok nice. We've been able to integrate the delay!

In [23]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,colorcet,scipy,biocrnpyler,jupyterlab
  

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 2.0.2
colorcet 2.0.2
scipy 1.4.1
biocrnpyler unknown
jupyterlab 1.2.6
